In [14]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from FileCounter import fileCount
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim

In [15]:
trainData = 'asl-alphabet/asl_alphabet_train/asl_alphabet_train'
testData = 'asl-alphabet/asl_alphabet_test/asl_alphabet_test'

In [16]:
train_transforms =  transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(trainData, transform=train_transforms)
test_dataset = datasets.ImageFolder(testData, transform=test_transforms)

In [17]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)


In [18]:

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 37 * 37, 128)
        self.fc2 = nn.Linear(128, 29)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 64 * 37 * 37)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [19]:
net = CNN()

# Step 4: Train the model
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')


[1,   200] loss: 3.229
[1,   400] loss: 2.675
[1,   600] loss: 2.251
[1,   800] loss: 1.944
[1,  1000] loss: 1.714
[1,  1200] loss: 1.508
[1,  1400] loss: 1.329
[1,  1600] loss: 1.176
[1,  1800] loss: 1.038
[1,  2000] loss: 0.979
[1,  2200] loss: 0.856
[2,   200] loss: 0.738
[2,   400] loss: 0.701
[2,   600] loss: 0.677
[2,   800] loss: 0.603
[2,  1000] loss: 0.549
[2,  1200] loss: 0.540
[2,  1400] loss: 0.490
[2,  1600] loss: 0.474
[2,  1800] loss: 0.456
[2,  2000] loss: 0.416
[2,  2200] loss: 0.416
[3,   200] loss: 0.333
[3,   400] loss: 0.299


KeyboardInterrupt: 